This jupyter notebook is used for rapid development.

In [70]:
import os
import yaml
import regex
import json




In [71]:
OPEN_API_FILE = "example_data/swagger_v2.yaml"
TAVERN_FILE_DIR = "example_data/functional"
TAVERN_FILES=[]
TAVERN_REGEX = "test_.*.tavern.yaml"
API_TARGET_URLS = ["{hsm_base_url}"] #use this to specify variables you care about!
TAVERN_CONFIG = "example_data/tavern_global_config.yaml"

# yaml.add_multi_constructor(u'!bool', lambda loader, suffix, node: None, Loader=yaml.SafeLoader)
# yaml.add_multi_constructor(u'!anything', lambda loader, suffix, node: None, Loader=yaml.SafeLoader)
# yaml.add_multi_constructor(u'!anybool', lambda loader, suffix, node: None, Loader=yaml.SafeLoader)

#if there are unknown tags, ignore them, return None; I am doing this is place of using safeloader which only uses known tags
class SafeLoaderIgnoreUnknown(yaml.SafeLoader):
    def ignore_unknown(self, node):
        return None
SafeLoaderIgnoreUnknown.add_constructor(None, SafeLoaderIgnoreUnknown.ignore_unknown)

# parse all the tavern test files

test_cases = []

for file in os.listdir(TAVERN_FILE_DIR):
    file_path = os.path.join(TAVERN_FILE_DIR, file)
    if os.path.isfile(file_path):
        if regex.search(TAVERN_REGEX, file) is not None:
            test_data = {}
            test_data ["file"] = file

            with open(file_path) as stream:
                #Im not using safe load, because I have a custom constructor to turn all unknown tags into Nones
                tavern_docs = yaml.load_all(stream, Loader=SafeLoaderIgnoreUnknown)
                for doc in tavern_docs:
                    test = doc["test_name"]
                    test_data["test_name"] = test
                    for stage in doc["stages"]:
                        test_stage = test + " - " + stage["name"]
                        test_data["stage"] = stage["name"]
                        test_data["request_url"] = stage["request"]["url"]
                        test_data["request_method"] = stage["request"]["method"].upper()
                        test_cases.append(test_data)
                        #test_case = (test_stage, stage["request"]["url"], stage["request"]["method"])
                        #print(test_case)
                    # for k,v in doc.items():
                    #     print(k,v)

print(json.dumps(test_cases, indent=2))

[
  {
    "file": "test_components.tavern.yaml",
    "test_name": "Ensure that we can call the Component Query API with node BMC xnames",
    "stage": "Ensure that we can conduct a query for a specific NodeBMC xname using the Component Query API",
    "request_url": "{hsm_base_url}/hsm/v2/State/Components/Query/{xname}",
    "request_method": "GET"
  },
  {
    "file": "test_components.tavern.yaml",
    "test_name": "Ensure that we can call the Component Query API with node BMC xnames",
    "stage": "Ensure that we can conduct a query for a specific NodeBMC xname using the Component Query API",
    "request_url": "{hsm_base_url}/hsm/v2/State/Components/Query/{xname}",
    "request_method": "GET"
  },
  {
    "file": "test_components.tavern.yaml",
    "test_name": "Ensure that we can call the Component Query API with node BMC xnames",
    "stage": "Ensure that we can conduct a query for a specific NodeBMC xname using the Component Query API",
    "request_url": "{hsm_base_url}/hsm/v2/St

In [72]:
# parse the swagger doc
api_methods = []
with open(OPEN_API_FILE) as stream:
    try:
        swagger = yaml.safe_load(stream)
        for path in swagger["paths"]:
            for method in swagger["paths"][path]:
                endpoint = {}
                endpoint["method"] = method.upper()
                endpoint["url"] = path
                api_methods.append(endpoint)
    except yaml.YAMLError as exc:
        print(exc)
        1/0

print(json.dumps(api_methods, indent=2))



[
  {
    "method": "GET",
    "url": "/service/ready"
  },
  {
    "method": "GET",
    "url": "/service/liveness"
  },
  {
    "method": "GET",
    "url": "/service/values"
  },
  {
    "method": "GET",
    "url": "/service/values/arch"
  },
  {
    "method": "GET",
    "url": "/service/values/class"
  },
  {
    "method": "GET",
    "url": "/service/values/flag"
  },
  {
    "method": "GET",
    "url": "/service/values/nettype"
  },
  {
    "method": "GET",
    "url": "/service/values/role"
  },
  {
    "method": "GET",
    "url": "/service/values/subrole"
  },
  {
    "method": "GET",
    "url": "/service/values/state"
  },
  {
    "method": "GET",
    "url": "/service/values/type"
  },
  {
    "method": "GET",
    "url": "/State/Components"
  },
  {
    "method": "POST",
    "url": "/State/Components"
  },
  {
    "method": "DELETE",
    "url": "/State/Components"
  },
  {
    "method": "GET",
    "url": "/State/Components/{xname}"
  },
  {
    "method": "PUT",
    "url": "/State/

In [74]:
#filter out any urls that dont have the expected URL string in them.
filtered_test_cases = []
for test_case in test_cases:
    for urls in API_TARGET_URLS:
        if urls in   test_case["request_url"]:
            filtered_test_cases.append(test_case)


In [75]:
with open(TAVERN_CONFIG) as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        logging.error(exc)
        exit(1)

#todo not sure I need this file?

In [79]:
# request_url = "{hsm_base_url}/hsm/v2/State/Components/{xname}"
# vs  url "/State/Components/{xname}" -> There is no guarantee that the variable is the same.  I think I want to convert all {.*} into a magic string like: {VARIABLE}
 #But need to do this AFTER the base url filtering...

for test_case in filtered_test_cases:
    test_case["request_url"] = regex.sub(r'\{(.*?)\}', "{VARIABLE}", test_case["request_url"])

print(filtered_test_cases)

api_methods

for endpoint in api_methods:
    endpoint["url"] = regex.sub(r'\{(.*?)\}', "{VARIABLE}", endpoint["url"])

print(api_methods)


[{'file': 'test_components.tavern.yaml', 'test_name': 'Ensure that we can call the Component Query API with node BMC xnames', 'stage': 'Ensure that we can conduct a query for a specific NodeBMC xname using the Component Query API', 'request_url': '{VARIABLE}/hsm/v2/State/Components/Query/{VARIABLE}', 'request_method': 'GET'}, {'file': 'test_components.tavern.yaml', 'test_name': 'Ensure that we can call the Component Query API with node BMC xnames', 'stage': 'Ensure that we can conduct a query for a specific NodeBMC xname using the Component Query API', 'request_url': '{VARIABLE}/hsm/v2/State/Components/Query/{VARIABLE}', 'request_method': 'GET'}, {'file': 'test_components.tavern.yaml', 'test_name': 'Ensure that we can call the Component Query API with node BMC xnames', 'stage': 'Ensure that we can conduct a query for a specific NodeBMC xname using the Component Query API', 'request_url': '{VARIABLE}/hsm/v2/State/Components/Query/{VARIABLE}', 'request_method': 'GET'}, {'file': 'test_com

In [81]:
#TODO start to build an endpoints map

endpoints = {}

for ep in api_methods:
    if ep in endpoints:
        1 + 1
    else:
        method = {}
        method [ep["method"]] = {}
        endpoints.setdefault(ep["url"], method)
        # endpoint = {}
        # endpoint[ep["method"]] = {}
        # endpoints[ep["url"]] = endpoint

print(endpoints)

TypeError: unhashable type: 'dict'

In [ ]:


from urllib.parse import urlparse


print(urlparse('//www.cwi.nl:80/%7Eguido/Python.html'))
print(urlparse('//www.cwi.nl:80/%7Eguido/Python.html?foo=true'))

#ParseResult(scheme='', netloc='www.cwi.nl:80', path='/%7Eguido/Python.html',            params='', query='', fragment='')
print(urlparse('www.cwi.nl/%7Eguido/Python.html'))
#ParseResult(scheme='', netloc='', path='www.cwi.nl/%7Eguido/Python.html', params='', query='', fragment='')
print(urlparse('help/Python.html'))
print(urlparse('{hsm_base_url}/hsm/v2/State/Components?type=NodeBMC'))

#ParseResult(scheme='', netloc='', path='help/Python.html', params='', query='', fragment='')